In [12]:
import requests
import psycopg2
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Get data into a dataframe

api_url = "https://api.coingecko.com/api/v3/exchange_rates"
response = requests.get(api_url)

data = response.json()
rates = data['rates']

tempdf = pd.DataFrame.from_dict(rates)
df = tempdf.T

print(df)

                            name  unit        value       type
btc                      Bitcoin   BTC          1.0     crypto
eth                        Ether   ETH       13.551     crypto
ltc                     Litecoin   LTC       216.78     crypto
bch                 Bitcoin Cash   BCH       153.85     crypto
bnb                 Binance Coin   BNB       58.803     crypto
...                          ...   ...          ...        ...
xdr   IMF Special Drawing Rights   XDR    12759.258       fiat
xag          Silver - Troy Ounce   XAG      764.615  commodity
xau            Gold - Troy Ounce   XAU        9.583  commodity
bits                        Bits  μBTC    1000000.0     crypto
sats                     Satoshi  sats  100000000.0     crypto

[61 rows x 4 columns]


In [3]:
# Connect to RDS Postgres database
conn = psycopg2.connect(
    database="crypto",
    user="jkamino",
    password="Tacoman123",
    host="data-5330-1.ckpcm5tpab0e.us-west-1.rds.amazonaws.com",
    port="5432")

cursor = conn.cursor() 

In [4]:
# Create Table if it doesn't exist
query = """
CREATE TABLE IF NOT EXISTS exchange_rates (
  exchange_id serial PRIMARY KEY,
  name VARCHAR(100) NOT NULL,
  unit VARCHAR(5) NOT NULL,
  value FLOAT, 
  type VARCHAR(20), 
  date TIMESTAMP  
);
"""

cursor.execute(query)
conn.commit()

In [5]:
# Loop through all items in df, ingest
for x in df.index:
    cursor.execute(f"""
                    INSERT INTO exchange_rates ( name, unit, value, type, date)
                    VALUES ('{df.loc[x]["name"]}', '{df.loc[x]["unit"]}', '{df.loc[x]["value"]}', '{df.loc[x]["type"]}', current_timestamp)
                    """)

    conn.commit()
    
    name = df.loc[x]["name"]
    
    print(f"{name} ingested")
    

Bitcoin ingested
Ether ingested
Litecoin ingested
Bitcoin Cash ingested
Binance Coin ingested
EOS ingested
XRP ingested
Lumens ingested
Chainlink ingested
Polkadot ingested
Yearn.finance ingested
US Dollar ingested
United Arab Emirates Dirham ingested
Argentine Peso ingested
Australian Dollar ingested
Bangladeshi Taka ingested
Bahraini Dinar ingested
Bermudian Dollar ingested
Brazil Real ingested
Canadian Dollar ingested
Swiss Franc ingested
Chilean Peso ingested
Chinese Yuan ingested
Czech Koruna ingested
Danish Krone ingested
Euro ingested
British Pound Sterling ingested
Hong Kong Dollar ingested
Hungarian Forint ingested
Indonesian Rupiah ingested
Israeli New Shekel ingested
Indian Rupee ingested
Japanese Yen ingested
South Korean Won ingested
Kuwaiti Dinar ingested
Sri Lankan Rupee ingested
Burmese Kyat ingested
Mexican Peso ingested
Malaysian Ringgit ingested
Nigerian Naira ingested
Norwegian Krone ingested
New Zealand Dollar ingested
Philippine Peso ingested
Pakistani Rupee inges

In [11]:
# Lets see if this worked
query = (f"""
SELECT *
FROM exchange_rates
""")

df_sql_table = pd.read_sql(query, conn,index_col='exchange_id')

df_sql_table.head(15)

C:\Users\JKami\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,name,unit,value,type,date
exchange_id,,,,,
1,Bitcoin,BTC,1.000,crypto,2022-12-06 20:33:20.708169
2,Ether,ETH,13.585,crypto,2022-12-06 20:33:21.561199
3,Litecoin,LTC,217.204,crypto,2022-12-06 20:33:22.236133
4,Bitcoin Cash,BCH,153.448,crypto,2022-12-06 20:33:23.026991
5,Binance Coin,BNB,58.848,crypto,2022-12-06 20:33:24.210938
6,EOS,EOS,17900.335,crypto,2022-12-06 20:33:24.693663
7,XRP,XRP,43571.184,crypto,2022-12-06 20:33:25.513030
8,Lumens,XLM,197916.040,crypto,2022-12-06 20:33:26.640043
9,Chainlink,LINK,2405.268,crypto,2022-12-06 20:33:27.151955
